In [63]:
from zipfile import ZipFile
import re
import json
import pandas as pd

In [64]:
zip_file = "./data.zip"
input_zip = ZipFile(zip_file)

In [65]:
fileNames = []
data = {}
pattern = 'data/[0-9]*.json'
for i in input_zip.namelist():
    if (re.match(pattern, i)):
        fileNames.append(i)
        data[i]=d = json.loads(input_zip.read(i))
len(fileNames)

1714

In [141]:
textToRemove = set(['TRANSCRIPT OF AUDIO FILE:','BEGIN TRANSCRIPT:', 'END TRANSCRIPT'])
patterns = ['.*\(.*\).*', '.*\[.*\].*']
patternToRemove = [' \(.*\)', ' \[.*\]']

tempCorT = ['.*CLIENT: ', '.*CLIENT:', 'THERAPIST: ','THERAPIST:']
CorTRep = ['CLIENT','CLIENT', 'THERAPIST', 'THERAPIST']


CorTSet = set(['CLIENT: ', 'THERAPIST: ', 'CLIENT:', 'THERAPIST:'])

for filename in fileNames:
    conversations = []
    if len(data[filename]['data']['conversation']) > 20:
        for s in data[filename]['data']['conversation']:
            if s not in textToRemove:
                text = s
                boolT = False
                for i,j in enumerate(patterns):
                    if re.match(j, s):
                        text = re.sub(patternToRemove[i], '', text)
                conversations.append(text)
        data[filename]['data']['newConversation'] = conversations
        
for filename in fileNames:
    conversations = []
    if 'newConversation' not in data[filename]['data']:
        continue
    
    for s in data[filename]['data']['newConversation']:
        text = s
        for i,v in enumerate(tempCorT):
            tMatch = re.match(v, text)
            if tMatch:
                if tMatch.group(0) not in CorTSet:
                    conversations = []
                    break
                text = re.sub(v, '', text)
                conversations.append([CorTRep[i], text ])
    data[filename]['data']['newConversation'] = conversations

for filename in fileNames:
    conversations = []
    if 'newConversation' not in data[filename]['data']:
        continue
    
    for s in data[filename]['data']['newConversation']:
        if s[1]: 
            conversations.append(s)
    data[filename]['data']['newConversation'] = conversations
    
for filename in fileNames:
    conversations = []
    if 'newConversation' not in data[filename]['data']:
        continue
        
    temp = False
    for i in range(len(data[filename]['data']['newConversation'])-1):
        if temp:
            temp = False
            continue
        curS = data[filename]['data']['newConversation'][i]
        nextS =  data[filename]['data']['newConversation'][i+1]
        tempS = curS
        
        
        if(curS[0] == nextS[0]):
            temp = True
            tempS[1] = tempS[1] + " " + nextS[1]
            
        conversations.append(tempS)
    data[filename]['data']['newConversation'] = conversations

In [142]:
columns=['fileName','condition', 'title', 'publication_year', 'main_conversation', 'conversation']
dfData = pd.DataFrame(columns=columns)
for index, filename in enumerate(fileNames):
    if 'newConversation' not in data[filename]['data']:
        continue
    if len(data[filename]['data']['newConversation']) == 0:
        continue
    if 'THERAPIST' == data[filename]['data']['newConversation'][0][0]:
        continue
    fileData = data[filename]
    condition = None
    if 'Presenting Condition' in fileData['Info']:
        condition = fileData['Info']['Presenting Condition']
    
    title = fileData['Info']['Abstract / Summary']
    
    publication_year = None
    if 'Publication Year' in fileData['Info']:
        publication_year = fileData['Info']['Publication Year']
    
    newConversation = fileData['data']['newConversation']
    
    conversation = fileData['data']['conversation']
    dfData.loc[len(dfData.index)] = [ filename, condition, title, publication_year, newConversation, conversation]
dfData.head()

,fileName,condition,title,publication_year,main_conversation,conversation
0,data/0.json,Anxiety,Client is concerned about managing his anxieti...,2013,"[[CLIENT, Hi there.], [THERAPIST, Hey.], [CLIE...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
1,data/10.json,Anxiety,Client talks about the cathartic affect that a...,2013,"[[CLIENT, So I'm sorry about Wednesday but I t...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
2,data/100.json,None,Client feels like he is finally starting to se...,2013,"[[CLIENT, Jesus, I got here, like, at 12:25.],...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
3,data/101.json,Anxiety,"Client has been feeling very stressed, overwhe...",2013,"[[CLIENT, I don't know, I guess I could. I mea...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
4,data/1017.json,Anxiety,Client discusses her reliance on other people ...,2014,"[[CLIENT, Good morning.], [THERAPIST, Hi.], [C...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."


In [146]:
for i, c in enumerate(dfData.main_conversation):
    cl = 0
    th = 0
    temp = False
    for s in c:
        if s[0] == 'CLIENT':
            cl+=1
        elif s[0] == 'THERAPIST':
            th+=1
            if (th == cl):
                temp = True
                
    if temp:
        print(i, cl, th)

0 48 48
1 96 95
2 107 105
3 57 54
4 64 56
5 76 68
6 43 36
7 54 43
8 49 45
9 78 77
10 43 36
11 76 67
13 59 52
14 49 41
15 63 53
16 48 43
17 47 42
18 48 40
19 134 134
21 41 30
22 43 32
23 57 57
24 16 15
25 28 27
26 104 103
27 24 24
28 70 64
29 26 25
30 19 19
32 16 15
33 36 35
34 17 16
35 43 43
36 44 43
37 21 21
38 74 74
39 27 27
40 36 35
41 21 21
42 15 15
43 48 47
44 59 58
45 27 27
46 79 74
47 38 38
48 82 82
49 31 31
50 37 37
51 46 46
52 38 38
53 135 135
54 52 52
55 36 35
56 23 22
57 11 11
58 130 128
59 19 18
60 23 22
61 52 52
62 22 22
63 71 71
64 25 25
65 22 21
66 21 20
67 37 36
68 22 21
69 30 30
70 33 32
71 101 99
72 90 88
73 87 86
74 48 42
75 94 93
76 60 59
77 59 58
78 99 99
79 44 44
80 62 61
81 122 120
82 81 81
83 31 30
84 46 45
85 39 38
86 38 37
87 52 52
88 97 96
89 76 75
90 64 63
91 28 27
92 26 25
93 53 52
94 28 27
95 31 32
96 28 27
97 105 103
98 78 77
99 63 62
100 47 47
101 65 49
103 82 81
104 33 33
105 40 39
106 71 70
107 41 41
108 33 32
109 31 30
110 30 29
111 42 43
113 116 115


In [147]:
dfData.loc[5].main_conversation

[['CLIENT',
  "Feeling a little better. It's good. Maybe it was that I hadn't come in here in awhile. I felt better after I left yesterday, or at least significantly better. Yeah my jaw hurts a little bit less, and I woke up early. That's good."],
 ['THERAPIST', 'Good change.'],
 ['CLIENT',
  "Yeah. I mean maybe it was like not that I feel so much better, but yeah now I feel back to a little bit back to where I was a little bit. And I think yeah maybe it's just yeah I was really sick and just rundown, and overwhelmed lately. I don't know. I think my grandmother's thing just got to  You know like yesterday I went to practice. It was great, like I didn't even think about my jaw, like when I was singing or. Had a great practice. It's definitely some kind of, or like the night before or whenever when I said I had that anxiety thing? But once I picked up my friend and we went "],
 ['THERAPIST',
  'I wonder what unique about when you say maybe it is not being here. That could mean a lot of d

In [121]:
dfData.loc[5].conversation

['TRANSCRIPT OF AUDIO FILE:',
 'BEGIN TRANSCRIPT:',
 'CLIENT: Good morning.',
 'THERAPIST: Hi.',
 "CLIENT: I was in a hurry today and didn't get (inaudible).",
 'CLIENT: Yeah, it\'s like the third time and I\'m like, \'do we have any like do I have a coffee mug that I can carry with me?" It\'s like I went to the cabinet and (inaudible).',
 'CLIENT: I ran into Dr. Hoffsteder (sp?) yesterday. She said some very nice things about you.',
 "THERAPIST: Oh, that's nice.",
 'CLIENT: Yeah. And she is -',
 "THERAPIST: I can't imagine why, but that's very (unclear).",
 'CLIENT: She seems very pleased about -',
 'THERAPIST: (inaudible).',
 "CLIENT: Yeah. (inaudible). She suggested I make some diet changes like not eat only pasta and bread. So I'm thinking about that.",
 "CLIENT: I feel like, well, it's the next thing to try. So, I should try it. But -",
 "CLIENT: I don't know. Except going on diets because I couldn't do it without shaming myself constantly. Like I don't know how to. I don't know h

In [148]:
compression_opts = dict(method='zip', archive_name='out.pkl')  
dfData.to_pickle('dataFinal.zip', compression=compression_opts) 

In [149]:
input_zip_pkl = ZipFile('./dataFinal.zip')
dfpkl =  pd.read_pickle(input_zip_pkl.open(input_zip_pkl.namelist()[0]))
dfpkl.head()

,fileName,condition,title,publication_year,main_conversation,conversation
0,data/0.json,Anxiety,Client is concerned about managing his anxieti...,2013,"[[CLIENT, Hi there.], [THERAPIST, Hey.], [CLIE...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
1,data/10.json,Anxiety,Client talks about the cathartic affect that a...,2013,"[[CLIENT, So I'm sorry about Wednesday but I t...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
2,data/100.json,None,Client feels like he is finally starting to se...,2013,"[[CLIENT, Jesus, I got here, like, at 12:25.],...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
3,data/101.json,Anxiety,"Client has been feeling very stressed, overwhe...",2013,"[[CLIENT, I don't know, I guess I could. I mea...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."
4,data/1017.json,Anxiety,Client discusses her reliance on other people ...,2014,"[[CLIENT, Good morning.], [THERAPIST, Hi.], [C...","[TRANSCRIPT OF AUDIO FILE:, BEGIN TRANSCRIPT:,..."


In [150]:
len(dfpkl)

824